In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import re
from scipy.interpolate import griddata
#from dedalus import public as de

In [2]:
def getproj(z, r_inner, r_outer):    
    if z < r_inner: 
        start_point = np.sqrt(r_inner**2 - z**2)
    else:
        start_point = 0
    end_point = np.sqrt(r_outer**2 - z**2)
    gridpoints = int(rscale.shape[0]/(r_outer-r_inner) * (end_point-start_point))
    
    #finding equidistant point with the same density as the original grid along the path of interpolation
    z_desired = np.asarray(np.linspace( start_point, end_point, num = gridpoints, endpoint = False))
    y_coord = np.asarray([z for i in z_desired])
    inter_coord = np.asarray([z_desired, y_coord]).T
    
    #padding and mesh
    phi_pad = np.append(phiscale[:], 2*np.pi)
    s_pad   = np.append(z_desired[:], end_point)
    s_mesh, phi_mesh = np.meshgrid(s_pad, phi_pad)
    
    return inter_coord, s_mesh, phi_mesh

In [3]:
def getplotdata(data, xykeys, datakeys):
    # this function creates a dictionary so I can find points more relevant with their coordinates
    datapoints = np.asarray([dat.ravel() for dat in data])
#    datadic = [dict(zip(xykeys, datpts)) for datpts in datapoints]
    datadic = dict(zip(xykeys, datapoints.T))
#    datas = np.asarray([[dic[key] for key in datakeys] for dic in datadic])
    datas = np.asarray([datadic[key] for key in datakeys]).T
    return datas

In [4]:
def plotnsave(val, s_mesh, phi_mesh, mydir, y):
    fig, ax = plt.subplots(subplot_kw=dict(polar=True))
    pcm = ax.pcolormesh(phi_mesh, s_mesh, val, cmap='RdYlBu_r')
    plt.colorbar(pcm)
    plt.savefig(mydir  + '/z%.3f_plot/%s_%04i.png' %(y ,"plot".format(0), counter), dpi=150)
    plt.close(fig)

In [5]:
#    if data.ndim == 5: np.linalg.norm(data, axis=4)
def makeplots(mydir, r_inner, r_outer, y):
    files = []
    
    #maybe not the best way, but I got lazy, and didn't want to not wrap everything in a function
    global data, rscale, thetascale, phiscale, counter    
    
    for file in os.listdir(mydir):
        if file.endswith(".h5"):
            files.append(os.path.join(mydir, file))
    files.sort(key=lambda f: int(re.sub('\D','', f)))

    counter = 0
    
    for file in files:
        df = h5py.File(file,"r")
        temp = df['tasks']['T']
        data = np.array(temp)

        if counter == 0:
            rscale = temp.dims[3][0][:]
            thetascale = temp.dims[2][0][:]
            phiscale = temp.dims[1][0][:]

            inter_coord, s_mesh, phi_mesh = getproj(y, r_inner, r_outer)

            xygrid = np.asarray([[r*np.sin(theta), r*np.cos(theta)] for theta in thetascale for r in rscale])
            inbetween = np.asarray([xy for xy in xygrid if (y - 1/26) <= xy[1] <= (y + 1/26)])
            # inbetween contains the relevent points. the smaller the set the faster the plot
            
            
            xykeys = list(map(str,xygrid))
            datakeys = list(map(str, inbetween))
            
            plot_dir = mydir + '/z%.3f_plot' %(y)
            if not os.path.exists(plot_dir):
                os.makedirs(plot_dir)


        for dats in data:
            datas = getplotdata(dats, xykeys, datakeys)

            grid_z0 = np.asarray([griddata(inbetween, dat, inter_coord, method='cubic') for dat in datas])
            grid_z0 -= np.mean(grid_z0, axis=0)

            plotnsave(grid_z0, s_mesh, phi_mesh, mydir, y)

            counter += 1   

In [ ]:
mydir = "data_Ek1e-6Ra300_L127N63_max2e-7"
r_inner = 7/13.
r_outer = 20./13.
z = 0.5*(r_outer-r_inner)+r_inner
makeplots(mydir, r_inner, r_outer, z)